# Test 11 (paper) - Experimento 7 (PT)

Entrenamiento de Mask R-CNN con dataset modificado para emular "Test 11"

### Hiperparametros
* **epoch = 100**
    * steps x epoch = 146 (lotes de imagenes)
    * batch = 2
* optimizador = SGD
* Funcion de perdida = SMOOTHL1LOSS
* Metrica de evaluacion = mAP (IoU >= 0.5)
* **Mini-mask shape: 28x28**
* **RPN anchor scales: (8, 16, 32, 64, 128)**
* Tasa de aprendizaje: 0.001
* **imagenes = 305**
    * entrenamiento 70% = 177
    * validacion 30% = 128
* etiquetas = 9140
* **resolucion = 1920 x 1080**
* etiquetas = bounding box formato VOC XML
* **numero de clases = 2 (arandano, arandano-maduro)**
* **data augmentation = false**


## Comprobar directorio principal

In [1]:
!pwd && ls -l

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport/model-training
total 1480
drwxr-xr-x 4 root root   4096 Aug  7 06:10 build
drwxr-xr-x 2 root root   4096 Aug  7 06:10 dist
drwxr-xr-x 2 root root   4096 Aug  8 03:15 mask_rcnn.egg-info
-rw-r--r-- 1 root root  67672 Sep 16 23:44 master-test_11.ipynb
-rw-r--r-- 1 root root  39459 Sep 16 23:53 master-test_11_fase_2-Copy1.ipynb
-rw-r--r-- 1 root root  67672 Sep 16 23:48 master-test_11_fase_2.ipynb
-rw-r--r-- 1 root root  30144 Sep 16 23:44 master-test_12.ipynb
-rw-r--r-- 1 root root  30144 Sep 16 22:02 master-test_12_fase_2.ipynb
-rw-r--r-- 1 root root 350337 Sep 15 00:38 master-test_5-fase_2.ipynb
-rw-r--r-- 1 root root 162983 Sep 15 02:23 master-test_5.ipynb
-rw-r--r-- 1 root root 350111 Sep 15 02:26 master-test_5_fase_2.ipynb
-rw-r--r-- 1 root root 110254 Sep 15 02:25 master-test_6.ipynb
-rw-r--r-- 1 root root 270148 Sep 15 15:04 master-test_6_fase_2.ipynb
drwxr-xr-x 4 root root   4096 Sep 13 07:49 mrcnn
drwxr-xr-x 3 root root   4096 Sep 15 

# Importar bibliotecas

In [2]:
# bibliotecas basicas
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# bibliotecas avanzadas 
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# bibliotecas mask rcnn 
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# biblioteca matplotlib 
import matplotlib.pyplot as plt

# bibliotecas numpy 
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# bibliotecas keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# ignorar alertas de elementos que seran descontinuados
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
#plt.show()

import imgaug.augmenters as iaa

# Fase 2 - Entrenamiento con dos clases y etiquetas de Bounding Box

En este entremamiento se utiliza un conjunto de datos simple con imágenes etiquetadas con cuadros delimitadores y una clase llamada 'Daño'. En la siguiente sección se encuentra el código para el entrenamiento del modelo. Se incluyen comentarios para describir mejor el flujo del programa.

In [3]:
class DatasetArandanos(Dataset):
    
    # la funcion load_dataset es usada para cargar el dataset de entrenamiento y test
    def load_dataset(self, dataset_dir, is_train=True):
        
        # se agrega una clase que se necesita para clasificar, en este caso arandano
        self.add_class("dataset", 1, "arandano")
        
        # se concatena dataset_dir con /images y /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train sera Verdadero si se esta entrenando el modelo y Falso si se esta testeando el modelo
        for filename in listdir(images_dir):
            
            # extraer image id
            image_id = filename[:-4] # se usa para omitir los últimos 4 caracteres: '.jpg' (en class_id.jpg)
            
            # si is_train es Verdadero se omiten todas las imágenes con id mayor que e iguales a 11074
            # aproximadamente el 80% del conjunto de datos es para entrenamiento
            
            if is_train and int(image_id) >= 11074 :
                print("image_id: ", image_id)
                continue
            
            # si is_train no es Verdadero se omiten todas las imágenes con id menores a 11074
            if not is_train and int(image_id) < 11074:
                continue
            
            # se declara la ruta de la imagen y la ruta de las etiquetas 
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # usando la función add_image se pasan image_id, image_path y ann_path para que la 
            # imagen actual se agregue al conjunto de datos para entrenamiento o prueba
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # funcino usada para extraer bouding boxes desde archivos etiquetados 
    def extract_boxes(self, filename):

        # se puede ver en las imágenes que estan etiquetadas, como se extraen los valores de ancho, alto y bndbox
        
        # <annotation>
        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>
        # </annotation>
        
        # para analizar los archivos .xml
        tree = ElementTree.parse(filename)
        
        # para obtener la raíz del archivo xml
        root = tree.getroot()
        
        # se agregan todas las coordenadas x, y en boxes para todas las instancias de un objeto
        boxes = list()
        
        # se encuentran todos los atributos con el nombre bndbox que existan para cada ground truth en la imagen
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        
        # extraer ancho y alto de la imagen
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # retorna boxes-> list, width-> int y height-> int 
        return boxes, width, height
    
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances

    # esta función llama al método extract_boxes y se usa para cargar una máscara para cada instancia en una imagen 
    # devuelve una máscara booleana con las siguientes dimensiones ancho * alto * instancias
    def load_mask(self, image_id):
        
        # info apunta al image_id actual 
        info = self.image_info[image_id]
        
        # se obtiene la ruta de anotación de image_id que es dataset_dir/annots/image_id.xml 
        path = info['annotation']
        
        # se llama al método extract_boxes (arriba) para obtener bndbox del archivo .xml
        boxes, w, h = self.extract_boxes(path)
        
        # se crea una cantidad de len(boxes) de mascaras de alto 'h' y ancho 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        # se agrega (append) el class_id 1 para arandano en nuestro caso a la variable
        class_ids = list()
        
        # se recorren todos los boxes y generamos máscaras (máscara de bndbox) y class id para cada instancia
        # las máscaras tendrán forma rectangular ya que hemos usado bndboxes para etiquetas
        # por ejemplo: si 2.jpg tiene tres objetos, tendremos las siguientes máscaras y class_ids.

        # 000000000 000000000 000001110 
        # 000011100 011100000 000001110
        # 000011100 011100000 000001110
        # 000000000 011100000 000000000
        #    1         1          1    <- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('arandano'))
        
        # retorna mascaras y class_ids como arreglo
        return masks, asarray(class_ids, dtype='int32')
    
    # esta funciones toma el image_id y retorna la ruta de la imagen 
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']


In [8]:
# damage configuration class, you can change values of hyper parameters here
class ConfigArandanos(Config):

    # nombre de la configuracion
    NAME = "arandano_cfg_test_11_fase_2_"    
    
    # clase arandano + clase background + 4 clases
    NUM_CLASSES = 1 + 1
    
    # pasos por epoch y confianza minima    # STEPS_PER_EPOCH = cantidad de lotes/batchs
    STEPS_PER_EPOCH = 152  

    # tasa de aprendizaje y momentum
    LEARNING_RATE=0.001
    LEARNING_MOMENTUM = 0.8
    
    # penalización de regularización
    WEIGHT_DECAY = 0.0001
    
    # el tamaño de la imagen está controlado por este parámetro
    IMAGE_MIN_DIM = 512
    
    # pasos de validación
    VALIDATION_STEPS = 50
    
    # número de regiones de interés generadas por imagen
    Train_ROIs_Per_Image = 200
    
    # escala de anclas RPN y proporciones (ratios) para encontrar la ROI
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)    # Longitud del lado del ancla cuadrada, en píxeles 
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]   # Proporciones de anclas por cada celda (ancho/alto). Un valor de 1 representa un ancla cuadrada y 0,5 es un ancla ancha 

    #DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0    
    DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

    IMAGES_PER_GPU = 2
    
    MINI_MASK_SHAPE = (28, 28)
    
ConfigArandanos().display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DEVICE                         /gpu:0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.8
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE 

### Entrenamiento

In [9]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [10]:
#pesos = 'mask_rcnn_coco.h5'
#pesos = 'mask_rcnn_damage_cfg_0049.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
pesos = 'arandano_cfg_test_5_fase_2_20220913T0749/mask_rcnn_arandano_cfg_test_5_fase_2__0300.h5'

conjunto_datos = 'customImages/test_11_fase_2'


In [11]:
# cargar dataset de entrenamiento
train_set = DatasetArandanos()
train_set.load_dataset(conjunto_datos, is_train=True)
train_set.prepare()

# cargar dataset de test 
test_set = DatasetArandanos()
test_set.load_dataset(conjunto_datos, is_train=False)
test_set.prepare()

# preparar la configuración llamando a la clase de configuración definida por el usuario
config = ConfigArandanos()

# definir el modelo
with tf.device(config.DEVICE):
    model = MaskRCNN(mode='training', model_dir='./', config=config)

# cargar pesos del modelo 
weights_path = pesos

# cargar los pesos del modelo
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

#augmentation = iaa.Sequential(iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sometimes(5/6,iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sequential([iaa.Fliplr(0.5), iaa.Flipud(0.5)])
#augmentation = iaa.Sometimes(5/6, iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sometimes(0.5, iaa.Fliplr(0.5), iaa.Flipud(0.5))

# start the training of model
# you can change epochs and layers (head or all)
#model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=100, layers='all', augmentation=augmentation)
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=100, layers='all')


image_id:  31102
image_id:  40099
image_id:  51320
image_id:  30377
image_id:  40414
image_id:  21130
image_id:  51422
image_id:  50469
image_id:  41732
image_id:  25544
image_id:  50458
image_id:  23863
image_id:  30990
image_id:  30470
image_id:  20318
image_id:  50440
image_id:  21295
image_id:  22345
image_id:  40390
image_id:  51286
image_id:  20782
image_id:  30676
image_id:  30000
image_id:  50135
image_id:  50912
image_id:  22783
image_id:  20150
image_id:  40346
image_id:  31246
image_id:  25750
image_id:  50983
image_id:  50607
image_id:  25690
image_id:  51398
image_id:  40999
image_id:  21150
image_id:  23719
image_id:  11117
image_id:  20425
image_id:  40253
image_id:  51082
image_id:  20250
image_id:  41406
image_id:  20585
image_id:  11076
image_id:  50393
image_id:  30851
image_id:  50958
image_id:  50857
image_id:  25640
image_id:  51057
image_id:  50710
image_id:  31284
image_id:  40213
image_id:  40960
image_id:  50386
image_id:  41762
image_id:  41383
image_id:  225

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/GatherV2_5:0", 

152/152 [==============================] - 362s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 2.8189 - rpn_class_loss: 0.4037 - rpn_bbox_loss: 1.1469 - mrcnn_class_loss: 0.1583 - mrcnn_bbox_loss: 0.6629 - mrcnn_mask_loss: 0.4471 - val_loss: 2.1843 - val_rpn_class_loss: 0.1794 - val_rpn_bbox_loss: 0.7007 - val_mrcnn_class_loss: 0.1927 - val_mrcnn_bbox_loss: 0.6450 - val_mrcnn_mask_loss: 0.4665
Epoch 2/100
152/152 [==============================] - 265s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 1.4478 - rpn_class_loss: 0.0600 - rpn_bbox_loss: 0.4664 - mrcnn_class_loss: 0.1731 - mrcnn_bbox_loss: 0.3132 - mrcnn_mask_loss: 0.4350 - val_loss: 2.0130 - val_rpn_class_loss: 0.1438 - val_rpn_bbox_loss: 0.6978 - val_mrcnn_class_loss: 0.2095 - val_mrcnn_bbox_loss: 0.5077 - val_mrcnn_mask_loss: 0.4542
Epoch 3/100
152/152 [==============================] - 273s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 1.1738 - rpn_class_loss: 0.0447 - rpn_bbox_loss: 0.2904 - mrcnn_class_loss: 0.1796 -

Epoch 21/100
152/152 [==============================] - 264s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.3928 - rpn_class_loss: 0.0109 - rpn_bbox_loss: 0.0222 - mrcnn_class_loss: 0.0433 - mrcnn_bbox_loss: 0.0492 - mrcnn_mask_loss: 0.2672 - val_loss: 2.9538 - val_rpn_class_loss: 0.3520 - val_rpn_bbox_loss: 0.8749 - val_mrcnn_class_loss: 0.8189 - val_mrcnn_bbox_loss: 0.4444 - val_mrcnn_mask_loss: 0.4636
Epoch 22/100
152/152 [==============================] - 256s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.3753 - rpn_class_loss: 0.0098 - rpn_bbox_loss: 0.0211 - mrcnn_class_loss: 0.0412 - mrcnn_bbox_loss: 0.0465 - mrcnn_mask_loss: 0.2567 - val_loss: 2.8535 - val_rpn_class_loss: 0.2979 - val_rpn_bbox_loss: 0.8394 - val_mrcnn_class_loss: 0.7112 - val_mrcnn_bbox_loss: 0.4392 - val_mrcnn_mask_loss: 0.5658
Epoch 23/100
152/152 [==============================] - 246s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.3652 - rpn_class_loss: 0.0101 - rpn_bbox_loss: 0.0219 - mrcnn_class

Epoch 41/100
152/152 [==============================] - 253s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.2191 - rpn_class_loss: 0.0058 - rpn_bbox_loss: 0.0104 - mrcnn_class_loss: 0.0194 - mrcnn_bbox_loss: 0.0230 - mrcnn_mask_loss: 0.1606 - val_loss: 3.2817 - val_rpn_class_loss: 0.4253 - val_rpn_bbox_loss: 0.8416 - val_mrcnn_class_loss: 0.9772 - val_mrcnn_bbox_loss: 0.4165 - val_mrcnn_mask_loss: 0.6210
Epoch 42/100
152/152 [==============================] - 256s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.2323 - rpn_class_loss: 0.0061 - rpn_bbox_loss: 0.0157 - mrcnn_class_loss: 0.0208 - mrcnn_bbox_loss: 0.0255 - mrcnn_mask_loss: 0.1641 - val_loss: 3.6326 - val_rpn_class_loss: 0.4169 - val_rpn_bbox_loss: 0.8726 - val_mrcnn_class_loss: 1.1972 - val_mrcnn_bbox_loss: 0.4721 - val_mrcnn_mask_loss: 0.6738
Epoch 43/100
152/152 [==============================] - 258s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.2183 - rpn_class_loss: 0.0059 - rpn_bbox_loss: 0.0148 - mrcnn_class

Epoch 61/100
152/152 [==============================] - 283s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.1610 - rpn_class_loss: 0.0034 - rpn_bbox_loss: 0.0119 - mrcnn_class_loss: 0.0136 - mrcnn_bbox_loss: 0.0180 - mrcnn_mask_loss: 0.1140 - val_loss: 4.3549 - val_rpn_class_loss: 0.6033 - val_rpn_bbox_loss: 0.8693 - val_mrcnn_class_loss: 1.4481 - val_mrcnn_bbox_loss: 0.4415 - val_mrcnn_mask_loss: 0.9926
Epoch 62/100
152/152 [==============================] - 283s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.1446 - rpn_class_loss: 0.0031 - rpn_bbox_loss: 0.0080 - mrcnn_class_loss: 0.0117 - mrcnn_bbox_loss: 0.0141 - mrcnn_mask_loss: 0.1076 - val_loss: 3.8855 - val_rpn_class_loss: 0.5701 - val_rpn_bbox_loss: 0.8567 - val_mrcnn_class_loss: 1.0767 - val_mrcnn_bbox_loss: 0.4266 - val_mrcnn_mask_loss: 0.9554
Epoch 63/100
152/152 [==============================] - 245s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.1412 - rpn_class_loss: 0.0035 - rpn_bbox_loss: 0.0082 - mrcnn_class

Epoch 81/100
152/152 [==============================] - 262s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.1338 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0104 - mrcnn_class_loss: 0.0107 - mrcnn_bbox_loss: 0.0152 - mrcnn_mask_loss: 0.0948 - val_loss: 4.0077 - val_rpn_class_loss: 0.4791 - val_rpn_bbox_loss: 0.8235 - val_mrcnn_class_loss: 1.2480 - val_mrcnn_bbox_loss: 0.4549 - val_mrcnn_mask_loss: 1.0022
Epoch 82/100
152/152 [==============================] - 246s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.1410 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0131 - mrcnn_class_loss: 0.0103 - mrcnn_bbox_loss: 0.0173 - mrcnn_mask_loss: 0.0974 - val_loss: 3.9270 - val_rpn_class_loss: 0.5980 - val_rpn_bbox_loss: 0.9070 - val_mrcnn_class_loss: 0.9979 - val_mrcnn_bbox_loss: 0.4087 - val_mrcnn_mask_loss: 1.0155
Epoch 83/100
152/152 [==============================] - 257s 2s/step - batch: 75.5000 - size: 2.0000 - loss: 0.1308 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0094 - mrcnn_class